In [1]:
import email
import re
from bs4 import BeautifulSoup

In [2]:

files=[None]*20
files[0]="/home/jovyan/work/dev/enron-playground/data/maildir/giron-d/all_documents/25."
files[1]="/home/jovyan/work/dev/enron-playground/data/maildir/giron-d/_sent_mail/7."
files[2]="/home/jovyan/work/dev/enron-playground/data/maildir/ruscitti-k/all_documents/59."
files[3]="/home/jovyan/work/dev/enron-playground/data/maildir/ruscitti-k/ces/5."
files[4]="/home/jovyan/work/dev/enron-playground/data/maildir/ruscitti-k/emailaddresses/22."
files[5]="/home/jovyan/work/dev/enron-playground/data/maildir/ruscitti-k/discussion_threads/78."
files[6]="/home/jovyan/work/dev/enron-playground/data/maildir/ruscitti-k/discussion_threads/315."
files[7]="/home/jovyan/work/dev/enron-playground/data/maildir/ruscitti-k/discussion_threads/153."
files[8]="/home/jovyan/work/dev/enron-playground/data/maildir/ruscitti-k/discussion_threads/119."
files[9]="/home/jovyan/work/dev/enron-playground/data/maildir/beck-s/deleted_items/13."
files[10]="/home/jovyan/work/dev/enron-playground/data/maildir/stclair-c/all_documents/846."
files[11]="/home/jovyan/work/dev/enron-playground/data/maildir/baughman-d/all_documents/176."
files[12]="/home/jovyan/work/dev/enron-playground/data/maildir/shackleton-s/all_documents/5210."
files[13]="/home/jovyan/work/dev/enron-playground/data/maildir/dean-c/inbox/790."
files[14]="/home/jovyan/work/dev/enron-playground/data/maildir/hyatt-k/deleted_items/458."
files[15]="/home/jovyan/work/dev/enron-playground/data/maildir/causholli-m/deleted_items/301."
files[16]="/home/jovyan/work/dev/enron-playground/data/maildir/campbell-l/all_documents/354."
files[17]="/home/jovyan/work/dev/enron-playground/data/maildir/hyvl-d/all_documents/1414."
files[18]="/home/jovyan/work/dev/enron-playground/data/maildir/forney-j/sent_items/158."
files[19]="/home/jovyan/work/dev/enron-playground/data/maildir/dasovich-j/all_documents/12700."


stop_regexes = [
    re.compile('----\s*Forwarded by'),
    re.compile('----\s*Original Message'),
    re.compile('_{20}'),
    re.compile('\*{20}'),
    re.compile('={20}'),
    re.compile('-{20}'),
    re.compile('\son \d{2}\/\d{2}\/\d{2,4} \d{2}:\d{2}:\d{2} (AM|PM)$', re.I),
    re.compile('\d{2}\/\d{2}\/\d{2,4} \d{2}:\d{2} (AM|PM)', re.I),
    re.compile('^\s?>?(From|To):\s?', re.I), 
    re.compile('LOG MESSAGES:',re.I),
    re.compile('=3D=3D',re.I),
    re.compile('Memo from.*on \d{2}\s(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?),', re.I),
    re.compile('Outlook Migration Team', re.I),
    re.compile('PERSON~', re.I),
    re.compile('>>>', re.I)
    
]

            
def get_msg_from_file(filename):
    #extract email message from a file
    with open(filename, "r") as f:
        contents = f.read()

    rtn = email.message_from_string(contents)
    return rtn


def log_msg(msg):
    #prints out an email
    headers = msg.items()
    print headers
    #print msg.keys()
    
    body = msg.get_payload()
    print '----clean body-----'
    print sanitize_string(body)
    print '----full body-----'
    print body


def apply_stop_regexes(body):
    #applies the regexes contained in the stop_regex_global
    rtn = ''
    #split the body of the message into lines
    for line in body.splitlines():
        #if we find one of the stop expressions, stop processing and return immediately
        for stop_regex in stop_regexes:
            if(stop_regex.search(line))!=None:
                return rtn
        rtn+=line
        #add the line breaks again
        rtn+='\n'
    return rtn

def convert_html(html):
    soup = BeautifulSoup(html)
    rtn = soup.get_text()
    return rtn

def translate_unwanted(body):
    #translates unwanted characters
    rtn = body
    unwanted = [("=20"," "),("=01,","'"),("=02",":"),("=09",":"),("=\n","\n"),("01=07"," ")]
    for (char, replacement) in unwanted:
        rtn = rtn.replace(char, replacement)
    return rtn
    
def sanitize_string(str):
    #performs several sanitations
    rtn = str
    rtn = convert_html(rtn)
    rtn = apply_stop_regexes(rtn)
    rtn = translate_unwanted(rtn)
    return rtn



def build_db_record_dict(msg):
    rtn={}
    #these are the interesting fields to buid a schema
    fields=['Message-ID', 'Date', 'From', 'To', 'Subject','X-From', 'X-To']

    for field in fields:
        if field in msg:
            rtn[field.lower()] = msg[field]

    body = msg.get_payload()
    body = sanitize_string(body)
    rtn['body'] = body
    return rtn

def main():
    for filename in files:
        msg = get_msg_from_file(filename)
        
        if filename==files[19]:
            log_msg(msg)
            print build_db_record_dict(msg)

            
main()



[('Message-ID', '<7444962.1075843503007.JavaMail.evans@thyme>'), ('Date', 'Fri, 18 May 2001 00:46:00 -0700 (PDT)'), ('From', 'venturewire@venturewire.com'), ('To', 'jdasovic@enron.com'), ('Subject', 'VentureWire, Friday, May 18, 2001'), ('Mime-Version', '1.0'), ('Content-Type', 'text/plain; charset=ANSI_X3.4-1968'), ('Content-Transfer-Encoding', 'quoted-printable'), ('X-From', 'VentureWire <venturewire@venturewire.com>'), ('X-To', 'jdasovic@ENRON.COM'), ('X-cc', ''), ('X-bcc', ''), ('X-Folder', '\\Jeff_Dasovich_June2001\\Notes Folders\\All documents'), ('X-Origin', 'DASOVICH-J'), ('X-FileName', 'jdasovic.nsf')]
----clean body-----
::?:
?:VentureWire
:Friday, May 18, 2001::?
?:CONTENTS| COMPANIES| INVESTORS| ADVERTISE| TELL A FRIEND:VentureWire 
Index
::'VentureWire
::today: 710.0 ? -0.41%:?
? ::
::
::top stories
::
::=  Online Deliverer WhyRunOut.com Acquires Over 90% of PDQuick 
::=  Red Herring Makes Third Round of Layoffs, Cuts 20%
::=  Vulcan Ventures Buys Small World Media
::=  My

/opt/conda/envs/python2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /opt/conda/envs/python2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


In [3]:
str = "../data/maildir/dasovich-j/all_documents/1711."
print str.split("maildir/")[1]


dasovich-j/all_documents/1711.
